In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-29 15:52:36.254244: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 15:52:36.280970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 15:52:36.280998: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 15:52:36.281024: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 15:52:36.290746: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-29 15:52:38.886984: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:52:38.931821: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:52:38.934043: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-29 15:52:39.009035: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:52:39.011127: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 15:52:39.012915: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.nasnet.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.NASNetLarge(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

343610240/343610240 [==============================] - 21s 0us/step


In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: stem_conv1, False
Layer: stem_bn1, False
Layer: activation, False
Layer: reduction_conv_1_stem_1, False
Layer: reduction_bn_1_stem_1, False
Layer: activation_1, False
Layer: activation_3, False
Layer: separable_conv_1_pad_reduction_left1_stem_1, False
Layer: separable_conv_1_pad_reduction_right1_stem_1, False
Layer: separable_conv_1_reduction_left1_stem_1, False
Layer: separable_conv_1_reduction_right1_stem_1, False
Layer: separable_conv_1_bn_reduction_left1_stem_1, False
Layer: separable_conv_1_bn_reduction_right1_stem_1, False
Layer: activation_2, False
Layer: activation_4, False
Layer: separable_conv_2_reduction_left1_stem_1, False
Layer: separable_conv_2_reduction_right1_stem_1, False
Layer: activation_5, False
Layer: separable_conv_2_bn_reduction_left1_stem_1, False
Layer: separable_conv_2_bn_reduction_right1_stem_1, False
Layer: separable_conv_1_pad_reduction_right2_stem_1, False
Layer: activation_7, False
Layer: reduction_add_1_stem_1, False
Layer: s

In [13]:
len(base_model.layers)

1039

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [15]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [16]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
#x = aug_layer(inputs)
x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'NasnetLarge')

model_1.summary()

Model: "NasnetLarge"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (  (None, 4032)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 4032)              16128     
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               

In [17]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [18]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [19]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [20]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 15, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/15


2024-07-29 15:54:00.632818: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-29 15:54:00.797763: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-29 15:54:02.227057: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x764aec003660 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-29 15:54:02.227076: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-29 15:54:02.236336: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-29 15:54:02.314608: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 8.7334 - accuracy: 0.4723
Epoch 1: val_loss improved from inf to 6.46049, saving model to Trained_Models/NasnetLarge
74/74 [==============================] - 63s 728ms/step - loss: 8.7334 - accuracy: 0.4723 - val_loss: 6.4605 - val_accuracy: 0.8497 - lr: 0.0010
Epoch 2/15
74/74 [==============================] - ETA: 0s - loss: 5.3132 - accuracy: 0.5464
Epoch 2: val_loss improved from 6.46049 to 4.57959, saving model to Trained_Models/NasnetLarge
74/74 [==============================] - 52s 705ms/step - loss: 5.3132 - accuracy: 0.5464 - val_loss: 4.5796 - val_accuracy: 0.8497 - lr: 0.0010
Epoch 3/15
74/74 [==============================] - ETA: 0s - loss: 3.7903 - accuracy: 0.5972
Epoch 3: val_loss improved from 4.57959 to 3.38280, saving model to Trained_Models/NasnetLarge
74/74 [==============================] - 53s 720ms/step - loss: 3.7903 - accuracy: 0.5972 - val_loss: 3.3828 - val_accuracy: 0.8497 - lr: 0.0010
Epoch 4/15
74

In [21]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 4s 130ms/step - loss: 0.9222 - accuracy: 0.7325


[0.9222232103347778, 0.7325443625450134]

In [22]:
base_model.trainable = True

In [23]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [24]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [25]:
start_epoch = 15
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "NasnetLarge"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (  (None, 4032)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 4032)              16128     
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               

In [26]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 16/25
74/74 [==============================] - ETA: 0s - loss: 0.9986 - accuracy: 0.7200
Epoch 16: val_loss did not improve from 0.83712
74/74 [==============================] - 27s 213ms/step - loss: 0.9986 - accuracy: 0.7200 - val_loss: 3.0628 - val_accuracy: 0.0425 - lr: 0.0010
Epoch 17/25
74/74 [==============================] - ETA: 0s - loss: 1.0095 - accuracy: 0.7209
Epoch 17: val_loss did not improve from 0.83712
74/74 [==============================] - 13s 171ms/step - loss: 1.0095 - accuracy: 0.7209 - val_loss: 8.4239 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 18/25
74/74 [==============================] - ETA: 0s - loss: 0.9584 - accuracy: 0.7315
Epoch 18: val_loss did not improve from 0.83712
74/74 [==============================] - 13s 171ms/step - loss: 0.9584 - accuracy: 0.7315 - val_loss: 2.3477 - val_accuracy: 0.0049 - lr: 0.0010
Epoch 19/25
74/74 [==============================] - ETA: 0s - loss: 0.9058 - accuracy: 0.7442
Epoch 19: val_loss did not improve fr

In [27]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 3s 116ms/step - loss: 1.2763 - accuracy: 0.5479


[1.276302456855774, 0.5479289889335632]

In [29]:
start_epoch = 25
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [30]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [28]:
start_epoch = 25
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 26/35
74/74 [==============================] - ETA: 0s - loss: 0.6569 - accuracy: 0.8179
Epoch 26: val_loss improved from 0.68436 to 0.51696, saving model to Trained_Models/NasnetLarge
74/74 [==============================] - 54s 736ms/step - loss: 0.6569 - accuracy: 0.8179 - val_loss: 0.5170 - val_accuracy: 0.9228 - lr: 3.0000e-04
Epoch 27/35
50/74 [===================>..........] - ETA: 2s - loss: 0.6290 - accuracy: 0.8131
Epoch 27: val_loss did not improve from 0.51696
74/74 [==============================] - 13s 175ms/step - loss: 0.6199 - accuracy: 0.8234 - val_loss: 3.6452 - val_accuracy: 0.1612 - lr: 3.0000e-04
Epoch 28/35
74/74 [==============================] - ETA: 0s - loss: 0.6426 - accuracy: 0.8225
Epoch 28: val_loss did not improve from 0.51696
74/74 [==============================] - 13s 170ms/step - loss: 0.6426 - accuracy: 0.8225 - val_loss: 6.8856 - val_accuracy: 0.1543 - lr: 3.0000e-04
Epoch 29/35
74/74 [==============================] - ETA: 0s - loss: 0.6279 

In [29]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 3s 115ms/step - loss: 0.7482 - accuracy: 0.7704


[0.7481857538223267, 0.7704141736030579]

In [31]:
start_epoch = 35

for layer in base_model.layers[-100:]:
    layer.trainable = True

In [32]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000075), metrics = ['accuracy'])
model_1.summary()

Model: "NasnetLarge"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 4032)        84916818  
                                                                 
 global_average_pooling2d (  (None, 4032)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 4032)              16128     
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               

In [33]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 36/45
74/74 [==============================] - ETA: 0s - loss: 0.7491 - accuracy: 0.7641
Epoch 36: val_loss improved from 0.51696 to 0.39615, saving model to Trained_Models/NasnetLarge
74/74 [==============================] - 72s 837ms/step - loss: 0.7491 - accuracy: 0.7641 - val_loss: 0.3962 - val_accuracy: 0.9268 - lr: 7.5000e-05
Epoch 37/45
74/74 [==============================] - ETA: 0s - loss: 0.6622 - accuracy: 0.8030
Epoch 37: val_loss did not improve from 0.39615
74/74 [==============================] - 14s 185ms/step - loss: 0.6622 - accuracy: 0.8030 - val_loss: 2.1527 - val_accuracy: 0.5519 - lr: 7.5000e-05
Epoch 38/45
74/74 [==============================] - ETA: 0s - loss: 0.6301 - accuracy: 0.8107
Epoch 38: val_loss did not improve from 0.39615
74/74 [==============================] - 14s 183ms/step - loss: 0.6301 - accuracy: 0.8107 - val_loss: 3.1047 - val_accuracy: 0.1790 - lr: 7.5000e-05
Epoch 39/45
74/74 [==============================] - ETA: 0s - loss: 0.6477 

In [34]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 3s 117ms/step - loss: 0.6941 - accuracy: 0.7834


[0.6940546631813049, 0.7834319472312927]

In [37]:
start_epoch = 45
for layer in base_model.layers[-150:]:
    layer.trainable = True

In [38]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [39]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 46/55
74/74 [==============================] - ETA: 0s - loss: 0.1190 - accuracy: 0.9856
Epoch 46: val_loss did not improve from 0.16067
74/74 [==============================] - 36s 107ms/step - loss: 0.1190 - accuracy: 0.9856 - val_loss: 0.2369 - val_accuracy: 0.9585 - lr: 2.5000e-05
Epoch 47/55
74/74 [==============================] - ETA: 0s - loss: 0.1114 - accuracy: 0.9894
Epoch 47: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1114 - accuracy: 0.9894 - val_loss: 0.4673 - val_accuracy: 0.8724 - lr: 2.5000e-05
Epoch 48/55
74/74 [==============================] - ETA: 0s - loss: 0.1195 - accuracy: 0.9886
Epoch 48: val_loss did not improve from 0.16067
74/74 [==============================] - 6s 83ms/step - loss: 0.1195 - accuracy: 0.9886 - val_loss: 0.2321 - val_accuracy: 0.9525 - lr: 2.5000e-05
Epoch 49/55
74/74 [==============================] - ETA: 0s - loss: 0.0976 - accuracy: 0.9936
Epoch 49: val_loss did not improv

In [40]:
start_epoch = 55
for layer in base_model.layers[-700:]:
    layer.trainable = True

In [41]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Nasnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (  (None, 1056)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 1056)              4224      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               541184    
                                                                 
 batch_normalization_1 (Bat  (None, 512)               2048 

In [ ]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 56/65
74/74 [==============================] - ETA: 0s - loss: 0.1236 - accuracy: 0.9814
Epoch 56: val_loss did not improve from 0.16067
74/74 [==============================] - 55s 148ms/step - loss: 0.1236 - accuracy: 0.9814 - val_loss: 0.3335 - val_accuracy: 0.9308 - lr: 2.5000e-05
Epoch 57/65
74/74 [==============================] - ETA: 0s - loss: 0.1372 - accuracy: 0.9809
Epoch 57: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 107ms/step - loss: 0.1372 - accuracy: 0.9809 - val_loss: 0.1625 - val_accuracy: 0.9763 - lr: 2.5000e-05
Epoch 58/65
74/74 [==============================] - ETA: 0s - loss: 0.1013 - accuracy: 0.9903
Epoch 58: val_loss did not improve from 0.16067
74/74 [==============================] - 8s 108ms/step - loss: 0.1013 - accuracy: 0.9903 - val_loss: 0.2233 - val_accuracy: 0.9594 - lr: 2.5000e-05
Epoch 59/65
74/74 [==============================] - ETA: 0s - loss: 0.0955 - accuracy: 0.9911
Epoch 59: val_loss did not impr

In [ ]:
model_1.evaluate(test_datagen)

In [38]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


OSError: No file or directory found at /home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet

In [ ]:
resnet.evaluate(val_datagen)